In [1]:
from pycocotools.coco import COCO
from pycocoevalcap.eval import COCOEvalCap
import matplotlib.pyplot as plt
import skimage.io as io
import pylab

pylab.rcParams["figure.figsize"] = (16, 7.5)

import json
from json import encoder
import numpy as np

encoder.FLOAT_REPR = lambda o: format(o, ".3f")

In [2]:
blip2_model = "blip2-opt-2.7b"
clip_models = ["ViT-32B", "ViT-16B", "RN50", "RN101"]
leaked_feature_layers = {
    "ViT-16B": [
        "vit-base", 
        # "vit-no-proj"
    ],
    "ViT-32B": [
        "vit-base", 
        # "vit-no-proj"
    ],
    "RN50": [
        "resnet-base",
        # "resnet-layer1",
        # "resnet-layer2",
        # "resnet-layer3",
        # "resnet-layer4",
    ],
    "RN101": [
        "resnet-base",
        # "resnet-layer1",
        # "resnet-layer2",
        # "resnet-layer3",
        # "resnet-layer4",
    ],
}

In [4]:
# run for all
# annFile = r"H:\Code\AutoDL_backup\datasets\image_caption_generation\flickr8k\annotations\captions_val.json"
annFile = "/root/autodl-tmp/datasets/image_caption_generation/flickr8k/annotations/captions_val.json"
coco = COCO(annFile)

loading annotations into memory...
Done (t=0.04s)
creating index...
index created!


In [7]:
dataset_name = "flickr8k"
clip_model = "RN50"
leaked_feature_layer = "resnet-layer1"
# add_noise = True
resFile = f"../../processed_results/results_{dataset_name}_{blip2_model}_{clip_model}_{leaked_feature_layer}_noise.json"
cocoRes = coco.loadRes(resFile)
# create cocoEval object by taking coco and cocoRes
cocoEval = COCOEvalCap(coco, cocoRes)

# evaluate on a subset of images by setting
# cocoEval.params['image_id'] = cocoRes.getImgIds()
# please remove this line when evaluating the full validation set
# cocoEval.params["image_id"] = cocoRes.getImgIds()

cocoEval.evaluate()

Loading and preparing results...
DONE (t=0.01s)
creating index...
index created!
tokenization...


PTBTokenizer tokenized 102870 tokens at 558347.59 tokens per second.
PTBTokenizer tokenized 11341 tokens at 148459.41 tokens per second.


setting up scorers...
computing Bleu score...


AssertionError: 

In [5]:
dataset_name = "flickr8k"

In [ ]:
for clip_model in clip_models:
    for leaked_feature_layer in leaked_feature_layers[clip_model]:
        print(f"Model: {blip2_model}, CLIP: {clip_model}, Layer: {leaked_feature_layer}")
        resFile = f"../../processed_results/results_{dataset_name}_{blip2_model}_{clip_model}_{leaked_feature_layer}.json"
        cocoRes = coco.loadRes(resFile)
        # create cocoEval object by taking coco and cocoRes
        cocoEval = COCOEvalCap(coco, cocoRes)

        # evaluate on a subset of images by setting
        # cocoEval.params['image_id'] = cocoRes.getImgIds()
        # please remove this line when evaluating the full validation set
        cocoEval.params["image_id"] = cocoRes.getImgIds()

        # evaluate results
        # SPICE will take a few minutes the first time, but speeds up due to caching
        cocoEval.evaluate()

        # print output evaluation scores
        for metric, score in cocoEval.eval.items():
            print("%s: %.3f" % (metric, score))

        # for metric in [
        #     "Bleu_1",
        #     "Bleu_2",
        #     "Bleu_3",
        #     "Bleu_4",
        #     "METEOR",
        #     "ROUGE_L",
        #     "CIDEr",
        #     "SPICE",
        # ]:
        #     scores = [eva[metric] for eva in cocoEval.evalImgs]
        #     print(f"{metric}: {scores}")
        #     plt.figure(figsize=(10, 8))
        #     plt.hist(scores)
        #     plt.title(f"Histogram of {metric} Scores", fontsize=20)
        #     plt.xlabel(f"{metric} score", fontsize=20)
        #     plt.ylabel("result counts", fontsize=20)
        #     # plt.show()
        #     plt.savefig(
        #         f"/root/Image2CaptionAttack/plot/figs/common_metrics/{metric}_{dataset_name}_{blip2_model}_{clip_model}_{leaked_feature_layer}.pdf",
        #         dpi=600,
        #         bbox_inches="tight",
        #     )

In [17]:
resFile = f"../../processed_results/results_flickr8k_{blip2_model}_RN50_resnet-layer2.json"
cocoRes = coco.loadRes(resFile)
# create cocoEval object by taking coco and cocoRes
cocoEval = COCOEvalCap(coco, cocoRes)

# evaluate on a subset of images by setting

# cocoEval.params['image_id'] = cocoRes.getImgIds()
# please remove this line when evaluating the full validation set
cocoEval.params["image_id"] = cocoRes.getImgIds()

# evaluate results
# SPICE will take a few minutes the first time, but speeds up due to caching
cocoEval.evaluate()

for metric, score in cocoEval.eval.items():
    print("%s: %.3f" % (metric, score))

Loading and preparing results...
DONE (t=0.01s)
creating index...
index created!
tokenization...


PTBTokenizer tokenized 12674 tokens at 169870.95 tokens per second.
PTBTokenizer tokenized 12276 tokens at 168186.98 tokens per second.


setting up scorers...
computing Bleu score...
{'testlen': 10277, 'reflen': 10506, 'guess': [10277, 9277, 8277, 7277], 'correct': [2249, 316, 63, 20]}
ratio: 0.978202931658007
Bleu_1: 0.214
Bleu_2: 0.084
Bleu_3: 0.038
Bleu_4: 0.019
computing METEOR score...
METEOR: 0.081
computing Rouge score...
ROUGE_L: 0.186
computing CIDEr score...
CIDEr: 0.194
computing SPICE score...


Parsing reference captions
Parsing test captions


SPICE evaluation took: 1.456 s
SPICE: 0.087
Bleu_1: 0.214
Bleu_2: 0.084
Bleu_3: 0.038
Bleu_4: 0.019
METEOR: 0.081
ROUGE_L: 0.186
CIDEr: 0.194
SPICE: 0.087


In [2]:
# 绘制所有Victim model指定的metric下的分布图
victim_models = {
    "ViT-16B": "ViT-16B",
    "ViT-32B": "ViT-32B",
    "RN50": "ResNet50",
    "RN101": "ResNet101",
    "mobilenetv2": "MobileNetV2",
    "mobilenetv3-small": "MobileNetV3-Small",
    "mobilenetv3-large": "MobileNetV3-Large",
}

leaked_feature_layers = {
    "ViT-16B": ["vit-base", "vit-no-proj"],
    "ViT-32B": ["vit-base", "vit-no-proj"],
    "RN50": [
        "resnet-base",
        "resnet-layer1",
        "resnet-layer2",
        "resnet-layer3",
        "resnet-layer4",
    ],
    "RN101": [
        "resnet-base",
        "resnet-layer1",
        "resnet-layer2",
        "resnet-layer3",
        "resnet-layer4",
    ],
    "mobilenetv2": [
        "mobilenet-base",
        "mobilenet-layer1",
        "mobilenet-layer-mid",
        "mobilenet-all-blocks",
    ],
    "mobilenetv3-small": [
        "mobilenet-base",
        "mobilenet-layer1",
        "mobilenet-layer-mid",
        "mobilenet-all-blocks",
    ],
    "mobilenetv3-large": [
        "mobilenet-base",
        "mobilenet-layer1",
        "mobilenet-layer-mid",
        "mobilenet-all-blocks",
    ],
}

In [3]:
metrics = [
    "Bleu_1",
    "Bleu_2",
    "Bleu_3",
    "Bleu_4",
    "METEOR",
    "ROUGE_L",
    "CIDEr",
    "SPICE",
]

In [4]:
blip2_model = "blip2-opt-2.7b"
dst_name = "flickr8k"

In [ ]:
annFile = "/root/autodl-tmp/datasets/image_caption_generation/flickr8k/annotations/captions_val.json"
coco = COCO(annFile)

eval_val_dict = {}

for victim_model in victim_models:
    vic_val_dict = {}
    leaked_feature_layer = leaked_feature_layers[victim_model][0]
    resFile = f"/root/Image2CaptionAttack/processed_results/results_{dst_name}_{blip2_model}_{victim_model}_{leaked_feature_layer}.json"
    cocoRes = coco.loadRes(resFile)
    # create cocoEval object by taking coco and cocoRes
    cocoEval = COCOEvalCap(coco, cocoRes)

    # evaluate on a subset of images by setting
    # cocoEval.params['image_id'] = cocoRes.getImgIds()
    # please remove this line when evaluating the full validation set
    cocoEval.params["image_id"] = cocoRes.getImgIds()

    # evaluate results
    # SPICE will take a few minutes the first time, but speeds up due to caching
    cocoEval.evaluate()
    for metric in metrics:
        scores = [eva[metric] for eva in cocoEval.evalImgs]
        vic_val_dict[metric] = scores
    eval_val_dict[victim_model] = vic_val_dict

In [9]:
victim_model = "ViT-16B"

annFile = "/root/autodl-tmp/datasets/image_caption_generation/flickr8k/annotations/captions_val.json"
coco = COCO(annFile)


eval_val_dict_leaked_feature_layer = {}

for leaked_feature_layer in leaked_feature_layers[victim_model]:
    resFile = f"/root/Image2CaptionAttack/processed_results/results_{dst_name}_{blip2_model}_{victim_model}_{leaked_feature_layer}.json"
    cocoRes = coco.loadRes(resFile)
    # create cocoEval object by taking coco and cocoRes
    cocoEval = COCOEvalCap(coco, cocoRes)

    val_dict = {}

    # evaluate on a subset of images by setting
    # cocoEval.params['image_id'] = cocoRes.getImgIds()
    # please remove this line when evaluating the full validation set
    cocoEval.params["image_id"] = cocoRes.getImgIds()

    # evaluate results
    # SPICE will take a few minutes the first time, but speeds up due to caching
    cocoEval.evaluate()
    for metric in metrics:
        scores = [eva[metric] for eva in cocoEval.evalImgs]
        val_dict[metric] = scores

    eval_val_dict_leaked_feature_layer[leaked_feature_layer] = val_dict

loading annotations into memory...
Done (t=0.04s)
creating index...
index created!
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
tokenization...


PTBTokenizer tokenized 12674 tokens at 190772.58 tokens per second.
PTBTokenizer tokenized 12112 tokens at 204252.90 tokens per second.


setting up scorers...
computing Bleu score...
{'testlen': 10106, 'reflen': 10506, 'guess': [10106, 9106, 8106, 7106], 'correct': [3116, 746, 216, 73]}
ratio: 0.961926518179996
Bleu_1: 0.296
Bleu_2: 0.153
Bleu_3: 0.084
Bleu_4: 0.049
computing METEOR score...
METEOR: 0.127
computing Rouge score...
ROUGE_L: 0.271
computing CIDEr score...
CIDEr: 0.535
computing SPICE score...


Parsing reference captions
Parsing test captions


SPICE evaluation took: 1.223 s
SPICE: 0.190
Loading and preparing results...
DONE (t=0.01s)
creating index...
index created!
tokenization...


PTBTokenizer tokenized 12674 tokens at 195430.09 tokens per second.
PTBTokenizer tokenized 12660 tokens at 210693.55 tokens per second.


setting up scorers...
computing Bleu score...
{'testlen': 10661, 'reflen': 10506, 'guess': [10661, 9661, 8661, 7661], 'correct': [2870, 613, 159, 44]}
ratio: 1.0147534742051194
Bleu_1: 0.269
Bleu_2: 0.131
Bleu_3: 0.068
Bleu_4: 0.037
computing METEOR score...
METEOR: 0.112
computing Rouge score...
ROUGE_L: 0.241
computing CIDEr score...
CIDEr: 0.375
computing SPICE score...


Parsing reference captions
Parsing test captions


SPICE evaluation took: 1.139 s
SPICE: 0.159
